In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/  # upload gson file to colab
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri

mkdir: cannot create directory ‘/root/.kaggle’: File exists
brain-tumor-classification-mri.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip 'brain-tumor-classification-mri.zip'

Archive:  brain-tumor-classification-mri.zip
replace Testing/glioma_tumor/image(1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install autokeras
import autokeras as ak

In [ ]:
img_height = 224
img_width = 224
train_dir = '/content/Training'
test_dir = '/content/Testing'
batch_size = 40

train_data = ak.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset= "training",
    seed= 123,
    image_size= (img_height, img_width),
    batch_size= batch_size
)

test_data = ak.image_dataset_from_directory(
    test_dir,
    subset= "testing",
    seed= 123,
    image_size= (img_height, img_width),
    batch_size= batch_size
)

Found 2870 files belonging to 4 classes.
Using 2296 files for training.
Found 394 files belonging to 4 classes.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.ClassificationHead()(input_node)

clf = ak.AutoModel(inputs= input_node, outputs= output_node, overwrite= False, max_trials= 1, objective= 'val_accuracy')

In [ ]:
import datetime
import tensorflow as tf
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# train model
clf.fit(train_data, epochs= 2, callbacks= [tensorboard_callback])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/2
58/58 [==============================] - 275s 5s/step - loss: 2.2797 - accuracy: 0.7252
Epoch 2/2
58/58 [==============================] - 261s 4s/step - loss: 0.9282 - accuracy: 0.8567


In [ ]:
print(clf.evaluate(test_data))

10/10 [==============================] - 45s 4s/step - loss: 3.1955 - accuracy: 0.6878
[3.1954901218414307, 0.6878172755241394]


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
import os
clf.export_model().save(os.path.join('content', "model.h5"))

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("/content/content/model.h5")

In [ ]:
predicted_y = model.predict(test_data)
print(predicted_y)